Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.675318
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 50: 1.842464
Minibatch accuracy: 12.5%
Validation accuracy: 40.2%
Minibatch loss at step 100: 1.275010
Minibatch accuracy: 62.5%
Validation accuracy: 59.3%
Minibatch loss at step 150: 0.274833
Minibatch accuracy: 87.5%
Validation accuracy: 74.0%
Minibatch loss at step 200: 1.006938
Minibatch accuracy: 62.5%
Validation accuracy: 73.8%
Minibatch loss at step 250: 0.753892
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 300: 0.831303
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 350: 0.551346
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 400: 0.738172
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.798960
Minibatch accuracy: 68.8%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.709146
Minibatch accuracy: 75.0%
Validation accuracy: 80.9%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(tf.nn.relu(conv + layer1_biases),
                            ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(tf.nn.relu(conv + layer2_biases),
                            ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.470100
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.394379
Minibatch accuracy: 50.0%
Validation accuracy: 53.0%
Minibatch loss at step 100: 0.885483
Minibatch accuracy: 75.0%
Validation accuracy: 74.0%
Minibatch loss at step 150: 0.131834
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 200: 0.827998
Minibatch accuracy: 68.8%
Validation accuracy: 79.3%
Minibatch loss at step 250: 0.563748
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 300: 1.063431
Minibatch accuracy: 68.8%
Validation accuracy: 81.6%
Minibatch loss at step 350: 0.518549
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
Minibatch loss at step 400: 0.600120
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
Minibatch loss at step 450: 0.668620
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Minibatch loss at step 500: 0.485468
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [32]:
batch_size = 64
patch_size = {
    'c1': 5,
    'c2': 5,
    'c3': 5
}
depth = {
    'c1': 6,
    'c2': 16,
    'c3': 120
}
num_hidden = 84

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # learning_rate
  global_step = tf.Variable(0)  # count the number of steps taken.
  tf_starter_learning_rate = tf.placeholder(tf.float32)
  tf_decay_steps = tf.placeholder(tf.float32)
  learning_rate = tf.train.exponential_decay(tf_starter_learning_rate,
                                             global_step, tf_decay_steps, 0.9999, staircase=False)  
    
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size['c1'], patch_size['c1'], num_channels, depth['c1']], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth['c1']]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size['c2'], patch_size['c2'], depth['c1'], depth['c2']], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth['c2']]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [patch_size['c3'], patch_size['c3'], depth['c2'], depth['c3']], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth['c3']]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [depth['c3'], num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') # 1@28x28 => 6@28x28
    hidden = tf.nn.max_pool(tf.nn.relu(conv + layer1_biases),
                            ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID') # 6@28x28 => 6@14x14
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID') # 6@14x14 => 16@10x10
    hidden = tf.nn.max_pool(tf.nn.relu(conv + layer2_biases),
                            ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID') # 16@10x10 => 16@5x5
    conv = tf.nn.conv2d(hidden, layer3_weights, [1, 1, 1, 1], padding='VALID') # 16@5x5 => 120@1x1
    hidden = tf.nn.relu(conv + layer3_biases) # 120@1x1 => 120@1x1
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [33]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                tf_starter_learning_rate : 0.05, tf_decay_steps : 1}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.953250
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.530450
Minibatch accuracy: 50.0%
Validation accuracy: 47.6%
Minibatch loss at step 100: 1.111885
Minibatch accuracy: 73.4%
Validation accuracy: 72.0%
Minibatch loss at step 150: 0.695377
Minibatch accuracy: 87.5%
Validation accuracy: 77.1%
Minibatch loss at step 200: 0.507727
Minibatch accuracy: 84.4%
Validation accuracy: 79.9%
Minibatch loss at step 250: 0.527393
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 300: 0.446986
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 350: 0.541637
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Minibatch loss at step 400: 0.507714
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.547237
Minibatch accuracy: 84.4%
Validation accuracy: 83.4%
Minibatch loss at step 500: 0.395290
Minibatch accuracy: 90.6%
Validation accuracy: 83.1%
M